# 101 Prepare net zero CO2 scenarios
In this notebook we will perform the extensions. The key idea is to keep emissions constant after net zero CO2 is achieved.

In [1]:
import pyam
import silicone.time_projectors as timeproj

from pathlib import Path

from tqdm.notebook import tqdm

<IPython.core.display.Javascript object>

## Read in the input dataframe
We will need the scenarios generated by the notebook `100_prepare_input_scenario_db.ipynb`

In [2]:
df = pyam.IamDataFrame(
    Path(
        '../data/100_scenarios.csv'
    )
)

pyam.core - INFO: Reading file ../data/100_scenarios.csv


Load the metadata

In [3]:
df.load_meta(
    Path(
        '../data/100_meta.xlsx'
    )
)

Fill in 2100 for na

In [4]:
df.meta['netzero|CO2'].fillna(2100, inplace=True)

## Data processing

Step 1: We first need to swap all dates for years

In [5]:
df_time_swap = (
    df
    .swap_time_for_year()
)

Step 2: Now, we need to split out the CO2 and non-CO2 emissions.

In [6]:
co2 = df_time_swap.filter(
    variable=[
        'AR6 climate diagnostics|Infilled|Emissions|CO2|Energy and Industrial Processes',
        'AR6 climate diagnostics|Infilled|Emissions|CO2|AFOLU'
    ]
)

In [7]:
non_co2 = df_time_swap.filter(
    variable='*Emissions|CO2*',
    keep=False
)

Step 3: Flat line the CO2 trajectories after net zero is reached.

In [8]:
flat_co2 = []

In [9]:
for index, _ in tqdm(
    co2.meta.iterrows()
):
    mod = index[0]
    scen = index[1]
    # Step 3.1 Filter for the model-scenario combination
    _scen_to_crunch = co2.filter(
        model=mod,
        scenario=scen
    )
    # Step 3.2: Identify the net zero CO2 year
    nz_year = int(
        _scen_to_crunch
        .meta['netzero|CO2']
        .values[0]
    )
    # Step 3.3; Further filter the dataframe between 2015 and net zero
    _scen_to_crunch = (
        _scen_to_crunch
        .filter(
            year=range(2015,nz_year+1)
        )
    )
    if nz_year == 2100:
        _compiled_results = _scen_to_crunch
    else:
        # Step 3.4: Perform the linear extension
        projector = timeproj.LinearExtender()
        projected_emissions = (
            projector
            .derive_relationship(
                variable=_scen_to_crunch.variable,
                gradient=0,
                times=range(nz_year+1, 2101)
            )(_scen_to_crunch)
        )
        _compiled_results = pyam.concat(
            [
                _scen_to_crunch,
                projected_emissions
            ]
        )
    # Step 3.5: Append this to the list of dataframes
    flat_co2.append(
        _compiled_results
    )

0it [00:00, ?it/s]

Step 4: Cast the results to a pyam dataframe

In [10]:
flat_co2_pyam = pyam.concat(
    flat_co2
)

In [11]:
flat_co2_pyam.variable

['AR6 climate diagnostics|Infilled|Emissions|CO2|AFOLU',
 'AR6 climate diagnostics|Infilled|Emissions|CO2|Energy and Industrial Processes']

Step 5: Append the non CO2 emissions

In [12]:
compiled_emissions = pyam.concat(
    [
        flat_co2_pyam,
        non_co2
    ]
)

Step 6: Rename the scenarios

In [13]:
for scen in compiled_emissions.scenario:
    compiled_emissions.rename(
        scenario={
            scen:f'{scen}_NZCO2'
        },
        inplace=True
    )

## Save out the data

In [14]:
compiled_emissions.to_csv(
    Path(
        '../data/101_scenarios.csv'
    )
)